In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import loompy
import velocyto as vcy
import logging
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.interpolate import interp1d
import cytograph as cg
import loompy
import numpy as np
import scipy.sparse as sparse
import torch
from torch.distributions.poisson import Poisson
from torch.autograd import Variable
from torch.nn import MSELoss
from tqdm import trange
from types import SimpleNamespace
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection

2019-03-06 11:44:36,257 INFO: ERROR: failed to import igraph; some clustering methods will not work
/Users/stelin/anaconda53/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
2019-03-06 11:44:38,666 DEBUG: backend module://ipykernel.pylab.backend_inline version unknown


In [ ]:
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

class State:
    def __init__(self, label: str = "", kind: str = "std", ap: str = "telencephalon", maturation: Tuple[str, int] = ("rgl", 0)) -> None:
        self.label = label
        self.kind = kind  # "std", "emphasize", "cycling"
        self.ap = ap  # "telencephalon", "diencephalon", "midbrain", "pons", "medulla", "cerebellum"
        self.dv = dv  # "dorsal", "ventral"
        self.maturation = maturation  # first element should be "rgl", "nbl", or "mature"
        self.xy = (0, 0)
        self.radius = 1

    def draw(self) -> PatchCollection:
        patches = []
        if self.kind == "std":
            patches.append(Circle(self.xy, self.radius, edgecolor="black", facecolor="white", linewidth=1))
        elif self.kind == "emphasize":
            patches.append(Circle(self.xy, self.radius, edgecolor="black", facecolor="white", linewidth=2))
        elif self.kind == "cycling":
            patches.append(Circle(self.xy, self.radius, edgecolor="black", facecolor="white", linewidth=2, linestyle="--"))
        return PatchCollection(patches)

class Trajectory:
    def __init__(self, states: List[State]) -> None:
        self.states = states
    
    def draw(self) -> PatchCollection:
        patches = []

def layout(trajectories, width = 50, height = 30) -> None:
    # Goal here is to determine the (x, y) position of every State object
    # Create an arena for the layout; an element of the arena is True iff a State is placed there
    arena = np.zeros(shape=(height, width), dtype="bool")
    